## Task

В качестве тестового задания мы предоставляем вам датасет online_shoppers_intention.csv, который содержит информацию о посетителях интернет-магазина и их поведении на сайте. Необходимо проанализировать данный датасет, определив какие действия могут привести к конверсии и создать классификатор на основе ваших гипотез.

В качестве ответа необходимо предоставить Jupyter Notebook с вашим анализом и модель машинного обучения, которая сможет предсказывать, приносят ли действия пользователя Revenue или нет.

## Imports

In [2]:
# Data
import pandas as pd
import numpy as np

# ML
import sklearn

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


## Data Analysis

In [3]:
dataframe = pd.read_csv('online_shoppers_intention.csv')
dataframe.sample(5)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
813,0,0.0,0,0.0,22,1310.500000,0.009524,0.028571,23.863593,0.0,Mar,2,2,1,1,Returning_Visitor,False,True
5459,0,0.0,0,0.0,2,7.000000,0.000000,0.100000,0.000000,0.0,June,2,2,7,1,Returning_Visitor,False,False
3393,0,0.0,1,19.0,27,1523.050000,0.000000,0.001852,22.128889,0.0,May,1,1,1,4,Returning_Visitor,False,True
1867,2,6.0,1,35.0,12,240.000000,0.014286,0.042857,0.000000,0.0,Mar,2,2,7,10,Returning_Visitor,False,False
7413,5,269.8,0,0.0,8,312.333333,0.000000,0.010000,0.000000,0.0,Nov,1,1,9,4,Returning_Visitor,False,False


In [4]:
dataframe.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000
